Imports and input file

In [1]:
import pandas as pd

df = pd.read_csv('BTC-USD_17.09.2014-14.03.2024.csv')

df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%y')

In [2]:
from datetime import datetime

def prompt_for_date(prompt_message, min_date=None, max_date=None, comparison_date=None, must_be_after=False):
    while True:
        date_input = input(prompt_message)
        try:
            date = datetime.strptime(date_input, "%d-%m-%Y")
            if min_date is not None and date < min_date:
                print("The date must not be before %s." % min_date.strftime("%d-%m-%Y"))
                continue
            if max_date is not None and date > max_date:
                print("The date must not be after %s." % max_date.strftime("%d-%m-%Y"))
                continue
            if comparison_date is not None:
                if must_be_after and date <= comparison_date:
                    print("The end date must be after the start date %s." % comparison_date.strftime("%d-%m-%Y"))
                    continue
            return date
        except ValueError:
            print("Invalid date format. Please use the format dd-mm-yyyy.")

min_allowed_start_date = datetime.strptime("17-09-2014", "%d-%m-%Y")
max_allowed_end_date = datetime.strptime("14-03-2024", "%d-%m-%Y")

start_date = prompt_for_date("Enter the start date (dd-mm-yyyy).", min_date=min_allowed_start_date)
end_date = prompt_for_date("Enter the end date (dd-mm-yyyy).", max_date=max_allowed_end_date, comparison_date=start_date, must_be_after=True)

def prompt_for_number(prompt_message, minimum=None, condition=lambda x: True):
    while True:
        try:
            number = float(input(prompt_message))
            if minimum is not None and number <= minimum:
                print("The value must be greater than %s." % minimum)
                continue
            if not condition(number):
                print("The input does not meet the required condition.")
                continue
            return number
        except ValueError:
            print("Please enter a valid number.")

initial_balance = prompt_for_number("Enter the initial balance of the trading bot wallet: ", minimum=0)
profit_percent = prompt_for_number("Enter the % profit (value must be >= 1): ", minimum=1.00) 

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

start_date_df_ascending_6month = pd.to_datetime('2020-10-01')
end_date_df_ascending_6month  = pd.to_datetime('2021-04-01')
start_date_df_descending_6month = pd.to_datetime('2021-10-01')
end_date_df_descending_6month  = pd.to_datetime('2022-09-01')
start_date_df_stagnation_6month = pd.to_datetime('2015-11-01')
end_date_df_stagnation_6month  = pd.to_datetime('2016-04-01')
start_date_df_combined_1year = pd.to_datetime('2021-10-01')
end_date_df_combined_1year  = pd.to_datetime('2022-10-01')

In [3]:
def prepare_and_export(df, file_name, initial_balance, initial_profit):
    df['Position'] = 0
    df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume', 'Position': 'position'}, inplace=True)
    df.insert(1, 'time', '00:00')
    df['date'] = df['date'].dt.strftime('%m/%d/%Y')
    df['date'] = pd.to_datetime(df['date'])
    df.drop(columns=['Adj Close'], inplace=True)
    df['initial_balance'] = initial_balance
    df['profit'] = initial_profit
    df.to_csv(file_name, index=False)

filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].copy()
filtered_df_ascending_6month = df[(df['Date'] >= start_date_df_ascending_6month) & (df['Date'] <= end_date_df_ascending_6month)].copy()
filtered_df_descending_6month = df[(df['Date'] >= start_date_df_descending_6month) & (df['Date'] <= end_date_df_descending_6month)].copy()
filtered_df_stagnation_6month = df[(df['Date'] >= start_date_df_stagnation_6month) & (df['Date'] <= end_date_df_stagnation_6month)].copy()
filtered_df_combined_1year = df[(df['Date'] >= start_date_df_combined_1year) & (df['Date'] <= end_date_df_combined_1year)].copy()

prepare_and_export(filtered_df, 'btcusd_tb2_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_ascending_6month, 'btcusd_tb2_asc_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_descending_6month, 'btcusd_tb2_dsc_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_stagnation_6month, 'btcusd_tb2_stg_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_combined_1year, 'btcusd_tb2_com_data.csv', initial_balance, profit_percent)

In [4]:
filtered_df['prevClose'] = filtered_df['close'].shift(1)
filtered_df['prevPrevClose'] = filtered_df['close'].shift(2)
filtered_df['signal'] = 0

filtered_df.loc[filtered_df['prevClose'] > filtered_df['prevPrevClose'], 'signal'] = 1
filtered_df.loc[filtered_df['prevClose'] < filtered_df['prevPrevClose'], 'signal'] = -1

for df in [filtered_df_ascending_6month, filtered_df_descending_6month, 
           filtered_df_stagnation_6month, filtered_df_combined_1year]:
    df['prevClose'] = df['close'].shift(1)
    df['prevPrevClose'] = df['close'].shift(2)
    df['signal'] = 0
    df.loc[df['prevClose'] > df['prevPrevClose'], 'signal'] = 1
    df.loc[df['prevClose'] < df['prevPrevClose'], 'signal'] = -1

In [5]:
import time

def trading_bot_2(balance, data, fee_percent, target_profit):
    trades = []
    holding = 0
    open_position_long = False
    open_position_short = False
    hold_count = 0
    initial_balance = balance
    initial_price = 0

    def log_trade(action, date, current_price):
        unix_date = int(time.mktime(date.timetuple()))
        trade = [unix_date, date, action, current_price, open_price, high_price, low_price, close_price, holding, balance]
        trades.append(trade)

    def calculate_fee(amount):
        return amount * fee_percent / 100

    for index, row in data.iterrows():
        date = row['date']
        open_price = row['open']
        close_price = row['close']
        low_price = row['low']
        high_price = row['high']

        if open_position_long or open_position_short:
            if open_position_long:
                current_profit = (holding * close_price - calculate_fee(holding * close_price)) - (holding * initial_price)
            else:
                current_profit = (holding * initial_price - calculate_fee(holding * close_price)) - (holding * close_price)

            if (open_position_long and current_profit >= target_profit) or (open_position_short and current_profit <= -target_profit):
                balance += current_profit
                action = 'CLOSE' if open_position_long else 'COVER'
                log_trade(action, date, close_price)
                open_position_long = open_position_short = False
                hold_count = 0
                continue
            else:
                hold_count += 1

        if not open_position_long and not open_position_short:
            if row['signal'] == 1 and balance > 0:
                open_position_long = True
                initial_price = open_price
                holding = (balance - calculate_fee(balance)) / open_price
                initial_balance = holding * initial_price
                log_trade('LONG', date, open_price)
            elif row['signal'] == -1 and balance > 0:
                open_position_short = True
                initial_price = open_price
                holding = (balance - calculate_fee(balance)) / open_price
                initial_balance = balance
                balance -= calculate_fee(balance)
                log_trade('SHORT', date, open_price)
            else:
                log_trade('NO SIGNAL', date, open_price)

    final_balance = trades[-1][-1] if trades else balance
    return final_balance, trades


In [6]:
print(f"Initial Balance: ${initial_balance}")

final_balance, trades = trading_bot_2(initial_balance, filtered_df, 1, profit_percent)
print(f"Final Balance Custom Data: ${final_balance}")
trades_df = pd.DataFrame(trades, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_trend_tb2_trades.csv', index=False)

final_balance_1, trades = trading_bot_2(initial_balance, filtered_df_ascending_6month, 1, profit_percent)
print(f"Final Balance ASC_6M: ${final_balance_1}")
trades_df = pd.DataFrame(trades, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_trend_tb2_asc_trades.csv', index=False)

final_balance_2, trades = trading_bot_2(initial_balance, filtered_df_descending_6month, 1, profit_percent)
print(f"Final Balance DSC_6M: ${final_balance_2}")
trades_df = pd.DataFrame(trades, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_trend_tb2_dsc_trades.csv', index=False)

final_balance_3, trades = trading_bot_2(initial_balance, filtered_df_stagnation_6month, 1, profit_percent)
print(f"Final Balance STG_6M: ${final_balance_3}")
trades_df = pd.DataFrame(trades, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_trend_tb2_stg_trades.csv', index=False)

final_balance_4, trades = trading_bot_2(initial_balance, filtered_df_combined_1year, 1, profit_percent)
print(f"Final Balance COM_1Y: ${final_balance_4}")
trades_df = pd.DataFrame(trades, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'position_size', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_trend_tb2_com_trades.csv', index=False)

Initial Balance: $100.0
Final Balance Custom Data: $2419.0124697367837
Final Balance ASC_6M: $215.30248331584963
Final Balance DSC_6M: $123.7065429718465
Final Balance STG_6M: $125.44931470581423
Final Balance COM_1Y: $123.7065429718465
